# Challenge 3 - High Refund

## 1️⃣ Intro
### 🕵️‍♀️ Investigate High Refund Listings

Hello, fellow SP-API experts!


I am writing to you on behalf of the 🪄ListingsWizard company. <br />
Thanks to your previous work, our company sales have been performing as expected. However, there seems to be signs of **unusually high** refund rate amongst our listings. <br />
Can you please investigate and recover the sales? My team has provided data access below to assist your investigation.


Yours,
ListingsWizard

---

_* The following scenario uses a mock server and mock data. No real data is used._

### Your Quest
1. Analyze data 📈
2. Identify root cause 👀
3. Submit a fix 🔧

On correct submission, you will receive a passcode. Raise your hand if you receive the passcode 🖐️

First person to receive the correct passcode will be the 🪄 **Wizard of Glory** 💫💫💫


## 2️⃣ Setup
### 🧰 Installing Selling Partner API SDK

The **Amazon Selling Partner API SDK** is an official Amazon library that simplifies integration with Amazon’s SP-API. It handles things like authentication, request signing, and response parsing — so you can focus on building your solution instead of dealing with low-level API mechanics.

With this SDK, you can easily access endpoints like Orders, Listings, Reports, Feeds, and **Buy Shipping** using clean, structured Python code.

📚 Learn more or explore the source on GitHub: https://github.com/amzn/selling-partner-api-sdk/tree/main


📦 To install the SDK, run the below block 👇

In [ ]:
pip install amzn-sp-api==1.2.0

---

### 🐛 Mocking the SDK


Luckily the Amazon Solution Architect team created a mock server that can replicate the API responses without having credentials.

Below you will find `mock_oauth_endpoint` and `mock_endpoint` variables that can be used to skip the credentials.

In [ ]:
from spapi.models.datakiosk_v2023_11_15 import GetDocumentResponse, CreateQueryResponse, GetQueriesResponse, Query
from spapi import SPAPIConfig, SPAPIClient, ApiException, QueriesApi, ListingsApi

# Configuration for mock SP-API
mock_oauth_endpoint = "http://18.212.254.115:80/auth/o2/token"
mock_endpoint = "http://18.212.254.115:80"

config = SPAPIConfig(
    client_id="Chris",
    client_secret="Khoury",
    refresh_token="FakeToken",
    region="NA"
)
client = SPAPIClient(config, endpoint=mock_endpoint, oauth_endpoint=mock_oauth_endpoint)

data_kiosk_api = QueriesApi(client.api_client)
listings_api = ListingsApi(client.api_client)
print("💻 API Client initiated")

---

## 3️⃣ Let's Data Kiosk!

### 📊 Visualize Your Economics Data

🧠 What This Shows - Let’s bring the numbers to life!

- You'll see a bar chart of refund rates.
- High-refund SKUs will be marked in red.
- Easy to tell which MSKUs are the problem.

Make sure you’ve run the cell above so `get_document_response` is available.!



In [ ]:
graphql_query_str = (
    "query { analytics_economics_2024_03_15 { economics( "
    "marketplaceIds: [\"ATVPDKIKX0DER\"], "
    "startDate: \"2025-06-01\", endDate: \"2025-06-05\", "
    "aggregateBy: { date: DAY, productId: MSKU } ) { "
    "msku startDate endDate sales { "
    "unitsOrdered unitsRefunded "
    "orderedProductSales { amount currencyCode } "
    "refundedProductSales { amount currencyCode } "
    "} } } }"
)

graphql_query = {"query": graphql_query_str}

# Tell Amazon to start processing your query
create_query_response = data_kiosk_api.create_query(body=graphql_query)

# Check the status of your query
get_query_response = data_kiosk_api.get_query(query_id=create_query_response.query_id)

# Now let's get the actual data!
get_document = data_kiosk_api.get_document(document_id=get_query_response.data_document_id)

# Visualize the data
import ast
import pandas as pd
import matplotlib.pyplot as plt

# Safely convert the string to a dictionary
parsed_document = ast.literal_eval(get_document.document_url)

# Extract the economics data
economics_data = parsed_document["data"]["analytics_economics_2024_03_15"]["economics"]


# Build DataFrame
df = pd.DataFrame([
    {
        "MSKU": item["msku"],
        "Units Ordered": item["sales"]["unitsOrdered"],
        "Units Refunded": item["sales"]["unitsRefunded"],
        "Ordered Sales ($)": item["sales"]["orderedProductSales"]["amount"],
        "Refunded Sales ($)": item["sales"]["refundedProductSales"]["amount"]
    }
    for item in economics_data
])

# Calculate refund rates
df["Refund Rate (%)"] = (df["Units Refunded"] / df["Units Ordered"]) * 100
df["$ Refund Rate (%)"] = (df["Refunded Sales ($)"] / df["Ordered Sales ($)"]) * 100

# Flag high refund rate listings
df["High Refund?"] = df["Refund Rate (%)"] > 30

# Display table
display(df.sort_values("Refund Rate (%)", ascending=False))

# Plot refund rates
plt.figure(figsize=(10, 6))
bars = plt.bar(df["MSKU"], df["Refund Rate (%)"], color=["red" if high else "green" for high in df["High Refund?"]])
plt.title("Refund Rate by MSKU", fontsize=14)
plt.ylabel("Refund Rate (%)")
plt.xlabel("MSKU")

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 1, f'{height:.1f}%', ha='center', fontsize=10)

plt.ylim(0, 100)
plt.tight_layout()
plt.show()


---

## 4️⃣ Troubleshooting: 

### 🫆 Clue

Looks like a few SKUs are experiencing **up to a 50% refund rate** — that’s a major red flag! Let's dig into the issue and uncover what might be causing these returns.

### 🕵️‍♂️ Step 1: Inspect the Listing 

Let’s start with the basics. Execute the code below to call `getListingsItems` operation for one of the high-refund SKUs. This will help you inspect listing issues.


In [ ]:
import json
get_listing = listings_api.get_listings_item('AMY7FKRUBY7XV', 
                                             'BICYCLE-GRAY-M',
                                             ['ATVPDKIKX0DER'], 
                                             included_data=['attributes', 'issues','summaries'])

print("🛍️ Here's your listing: \n", json.dumps(get_listing.to_dict(), indent=4, default=str))

#### 🔎 New Clue

✅ No ERROR severity issues found! So what’s causing the high refund rate?

### 📊 Step 2: Check Seller Performance
  
  
  Our team just pulled a `GET_V2_SELLER_PERFORMANCE_REPORT` report. It might be relevant to the investigation!

<details>
<summary><strong>Click to see <code>GET_V2_SELLER_PERFORMANCE_REPORT</code> report</strong></summary>
    
```
{
  "accountStatuses": [
    {
      "marketplaceId": "ATVPDKIKX0DER",
      "status": "AT_RISK"
    }
  ],
  "performanceMetrics": [
    {
      "lateShipmentRate": {
        "reportingDateRange": {
          "reportingDateFrom": "2025-05-27T00:00:00Z",
          "reportingDateTo": "2025-06-25T23:59:59Z"
        },
        "status": "AT_RISK",
        "targetValue": 0.04,
        "targetCondition": "LESS_THAN",
        "orderCount": 5,
        "lateShipmentCount": 2,
        "rate": 0.4
      },
      "lateShipmentRateList": [
        {
          "reportingDateRange": {
            "reportingDateFrom": "2025-06-16T00:00:00Z",
            "reportingDateTo": "2025-06-25T23:59:59Z"
          },
          "status": "AT_RISK",
          "targetValue": 0.04,
          "targetCondition": "LESS_THAN",
          "orderCount": 5,
          "lateShipmentCount": 2,
          "rate": 0.4
        }
      ],
      "invoiceDefectRate": {},
      "orderDefectRate": {},
      "onTimeDeliveryRate": {},
      "unitOnTimeDeliveryRate": {},
      "validTrackingRate": {},
      "preFulfillmentCancellationRate": {},
      "warningStates": [],
      "accountHealthRating": {},
      "listingPolicyViolations": {},
      "productAuthenticityCustomerComplaints": {},
      "productConditionCustomerComplaints": {},
      "productSafetyCustomerComplaints": {},
      "receivedIntellectualPropertyComplaints": {},
      "restrictedProductPolicyViolations": {},
      "suspectedIntellectualPropertyViolations": {},
      "foodAndProductSafetyIssues": {},
      "customerProductReviewsPolicyViolations": {},
      "otherPolicyViolations": {},
      "documentRequests": {},
      "policyViolationWarnings": {},
      "marketplaceId": "ATVPDKIKX0DER"
    }
  ]
}
```

We see a `lateShipmentRate` of 0.4! 🤦‍♀️ 

⚠️ This means some orders have not been shipped on time — a potential driver behind returns.

```
"lateShipmentRate": {
        "reportingDateRange": {
          "reportingDateFrom": "2025-05-27T00:00:00Z",
          "reportingDateTo": "2025-06-25T23:59:59Z"
        },
        "status": "AT_RISK",
        "targetValue": 0.04,
        "targetCondition": "LESS_THAN",
        "orderCount": 5,
        "lateShipmentCount": 2,
        "rate": 0.4
```

</details>




## 5️⃣ Resolution: 


### 🧐 Analysis


### 🧠 Step 3: Inspect the getListingsItem Response

There is no `lead_time_to_ship_max_days` attribute. But according to our company’s internal records, this product type is supposed to have a **10-day lead time**. 🚚 Why would this happen? 

> Note:
> _Lead time to ship is the number of business days a seller takes to pick, pack and ship a Merchant Fulfilled Network (MFN) order after receiving it. Amazon allows sellers to set the handling time to the Same-day(0-day), 1-day, 2-day or more, depending on their fulfilment capabilities. To learn more, check this 👉 [article](https://github.com/amzn/selling-partner-api-samples/discussions/124)_

```
"fulfillment_availability": [
            {
                "fulfillment_channel_code": "DEFAULT",
                "quantity": 3
            }
        ],
```

### 📂 Step 4: Investigate Historical Submission

Our team pulled a historical patchListingsItem submission for this SKU. Let’s see what was submitted earlier for this SKU:

Can you observe and identify why the lead_time_to_ship_max_days could have been wrong? 


<details>
<summary><strong>Click to see Historical Submission</strong></summary>
    
```
{
    "productType": "PRODUCT",
    "patches": [
        {
            "op": "replace",
            "path": "/attributes/fulfillment_availability",
            "value": [
                {
                    "fulfillment_channel_code": "DEFAULT",
                    "quantity": 4
                }
            ]
        }
    ]
}
```
<br />
🧩 Can you identify what was wrong with this submission?

➡️ The previous patch overwrote the `lead_time_to_ship_max_days` because it wasn’t included in the payload. That’s likely how it got removed from the listing.

</details>


### Future Submissions

### 🛠️ Step 5: Fix

Our Shipping team has manually changed `lead_time_to_ship_max_days` back to 10 days. But we need to make sure we don't accidentally override the value again in the future. 

The team now needs to update just the inventory to 4. Can you help come up with a request to update inventory without providing and overwritting `lead_time_to_ship_max_days` attribute?

<details>
<summary><strong>💡 Click to see HINT 1</strong></summary>
    
_There is a [new merge operation](https://developer-docs.amazon.com/sp-api/changelog/update-listings-items-v2021-08-01-api-and-json_listing_feed-now-include-a-json-patch-merge-operation) introduced into the Listings APIs. You no longer need to pass optional field `lead_time_to_ship_max_days` when updating inventory only._

</details>

<details>
<summary><strong>💡 Click to see HINT 2</strong></summary>
    
_[Tutorial: Merge a listing](https://developer-docs.amazon.com/sp-api/docs/listings-items-api-v2021-08-01-use-case-guide#tutorial-merge-a-listing)_

</details>


In [ ]:
body = {
    "productType": "PRODUCT",
    "patches": [
        {
            "op": # provide value,
            "path": "/attributes/fulfillment_availability",
            "value": [
                {
                    "fulfillment_channel_code": "DEFAULT",
                    # provide value
                }
            ]
        }
    ]
}
patch_listing = listings_api.patch_listings_item('AMY7FKRUBY7XV', 'BICYCLE-GRAY-M',['ATVPDKIKX0DER'], body)
print("💾 Submission result: \n", json.dumps(patch_listing.to_dict(), indent=4, default=str))